# Building a Multi Agent Collaboration - with LangGraph

## Table of contents
- [Introduction](#introduction)
- [Prerequisites](#prerequisites)
- [Specialized Agents](#specialized-agents)
- [Summary](#summary)

## Introduction

In the previous lab we introduced the **LangGraph** framework, how to create agents and add memory to it. In this lab we will build on single agent and create multiple agents

## Objectives

By the end of this notebook, you will:
1. Understand how to create and connect multiple agents using LangGraph
2. Learn how to define agent roles and responsibilities
3. Implement a multi-agent system that can handle account enquiries, process new applications and provide FAQ
4. Demonstrate how inter agent communication and handover occurs


## Prerequisites

Before starting this notebook, ensure you have:
- Familiarity with LangChain concepts (optional but helpful)
- Basic understanding of Python and Jupyter notebooks
- AWS account with access to Amazon Bedrock and SageMaker


## Setup
Start by installing some of the required packages, including LangChain for pre-built tool components, LangGraph for agent workflows, and other necessary libraries.

In [ ]:
%pip uninstall boto3 botocore awscli --yes

In [ ]:
%pip install -r requirements.txt -q --force-reinstall

<div class="alert alert-block alert-info">
<b>Important:</b> restart the kernel before proceeding with the next cells. You can use the restart icon on the top of this notebook or  uncomment below cell and execute
</div>

In [ ]:
# restart kernel for packages to take effect
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

Import the following modules

In [ ]:
from typing import TypedDict, List, Annotated, Union
from langchain_aws import ChatBedrock, ChatBedrockConverse
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent, ToolNode,tools_condition
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, AnyMessage, SystemMessage, ToolMessage
from langgraph.graph.message import add_messages
from langgraph.graph import MessagesState, StateGraph, START, END
import os
import boto3
from IPython.display import display,Image
import random
from textwrap import dedent
from datetime import datetime, timedelta
from langgraph.checkpoint.memory import InMemorySaver
from datetime import datetime, timedelta
import certifi
 



Next, we setup the LLMs to be used for this lab. Although you will use these specific models for this lab, LangChain also supports models from other [providers](https://python.langchain.com/docs/integrations/chat/#featured-providers)


When setting up an LLM model, you can also set model parameters. In this case, we set `temperature` to 0.7 to balance creativity with accuracy. A higher temperature (closer to 1.0) would produce more creative but potentially less accurate responses, while a lower temperature (closer to 0) would produce more deterministic responses.

If you're running this notebook locally, set the AWS_PROFILE

In [ ]:
# os.environ['AWS_PROFILE']='stcgenai'


If you are getting certificate errors, you can use the following code block to explicitly add the certificate path

In [ ]:

# # Get the path to the certificate bundle
# cert_path = certifi.where()
# print(f"Using certificate bundle at: {cert_path}")

# # Set environment variable
# os.environ['AWS_CA_BUNDLE'] = cert_path

## Specialized Agents:


#### 1. Existing Mortgage Assistant
- **Role**: Manages existing customer mortgage accounts
- **Tools**: `get_mortgage_status` function
- **Expertise**: Account balances, interest rates, payment schedules, maturity dates

#### 2. Mortgage Application Agent
- **Role**: Handles new mortgage applications
- **Tools**: 
  - `get_mortgage_app_doc_status`
  - `get_application_details`
  - `get_mortgage_rate_history`
- **Expertise** : Tracks document submission status, retrieves application information with status, provide historical interest rates

#### 3. General Mortgage Questions Agent
- **Role**: Handles conceptual mortgage questions
- **Tools**: Knowledge Base access
- **Expertise**: Refinancing concepts, mortgage type comparisons (15-year vs 30-year)




Here is the code we have built in the previous lab for creating the first agent `Existing Mortgage Assistant`.

Depending on the model availability and the region  you are running this lab, change the agent_foundation_model[] and region_name before instantiating the boto client

In [ ]:
class AgentState(MessagesState):
    pass


agent_foundation_model = [
    'us.anthropic.claude-3-5-haiku-20241022-v1:0'
    ]

# Create a session with your profile
bedrock_client = boto3.client('bedrock-runtime')

# bedrock_client = boto3.client('bedrock-runtime',region_name='us-west-2')

model = init_chat_model(
    agent_foundation_model[0],
    model_provider="bedrock_converse",
    temperature=0.7,
    client=bedrock_client
)

memory = InMemorySaver()




In addition to the existing mortgage tools we will add few more tools -


In [ ]:
def get_mortgage_details(customer_id: str) -> str:
    """
Retrieves the mortgage status for a given customer ID. Returns an object containing 
details like the account number, 
outstanding principal, interest rate, maturity date, number of payments remaining, due date of next payment, 
and amount of next payment."""
    return {
        "account_number": customer_id,
        "outstanding_principal": 150599.25,
        "interest_rate": 8.5,
        "maturity_date": "2030-06-30",
        "original_issue_date": "2021-05-30",
        "payments_remaining": 72,
        "last_payment_date": str(datetime.today() - timedelta(days=14)).split(' ')[0],
        "next_payment_due": str(datetime.today() + timedelta(days=14)).split(' ')[0],
        "next_payment_amount": 1579.63
    }


def get_application_details(customer_id: str) -> str:
    """Retrieves the details about an application for a new mortgage. The function takes a customer ID, but it is purely optional. 
    The function implementation can retrieve it from session state instead. 
    Details include the application ID, application date, application status, application type, application amount, application tentative rate, and application term in years."""
    return {
        "customer_id": customer_id,
        "application_id": "998776",
        "application_date": datetime.today() - timedelta(days=35), # simulate app started 35 days ago
        "application_status": "IN_PROGRESS",
        "application_type": "NEW_MORTGAGE",
        "application_amount": 750000,
        "application_tentative_rate": 5.5,
        "application_term_years": 30,
        "application_rate_type": "fixed"
    }


def get_mortgage_rate_history(day_count: int=30, type: str="15-year-fixed"):
    """Retrieves the history of mortgage interest rates going back a given number of days, defaults to 30. History is returned as a list of objects, where each object contains the date and the interest rate to 2 decimal places."""
    BASE_RATE=6.00

    RATE_MIN_15=38
    RATE_MAX_15=48

    RATE_MIN_30=RATE_MIN_15 + 80
    RATE_MAX_30=RATE_MAX_15 + 80
    
    # print(f"getting rate history for: {day_count} days, for type: {type}...")
    # generate the last 7 working day dates starting with yesterday
    today = datetime.today()
    history_count = 0
    rate_history = []

    if type == "30-year-fixed":
        RATE_MIN = RATE_MIN_30
        RATE_MAX = RATE_MAX_30
    else:
        RATE_MIN = RATE_MIN_15
        RATE_MAX = RATE_MAX_15

    for i in range(int(day_count*1.4)):
        if history_count >= day_count:
            break
        else:
            day = today - timedelta(days=i+1)
            which_day_of_week = day.weekday()
            if which_day_of_week < 5:
                history_count += 1
                _date = str(day.strftime("%Y-%m-%d"))
                _rate = f"{BASE_RATE + ((random.randrange(RATE_MIN, RATE_MAX))/100):.2f}"
                rate_history.append({"date": _date, "rate": _rate})

    return rate_history


def get_mortgage_app_doc_status(customer_id: str):
    """
    Retrieves the list of required documents for a mortgage application in process, along with their respectiv statuses (COMPLETED or MISSING).
    The function takes a customer ID, but it is purely optional. The funciton implementation can retrieve it from session state instead. 
    This function returns a list of objects, where each object represents a required document type. 
    The required document types for a mortgage application are: proof of income, employment information, proof of assets, and credit information. Each object in the returned list contains the type of the required document and its corresponding status."""
    return [
        {
            "type": "proof_of_income",
            "status": "COMPLETED"
        },
        {
            "type": "employment_information",
            "status": "MISSING"
        },
        {
            "type": "proof_of_assets",
            "status": "COMPLETED"
        },
        {
            "type": "credit_information",
            "status": "COMPLETED"
        }
    ]

Before we create agents, we need to understand the multi agent collaboration concepts. 


For this lab, we will concentrate on SWARM architecture pattern. LangGraph has a built pattern to use the SWARM architecture




In [ ]:
from langgraph_swarm import create_handoff_tool, create_swarm


we will create two hand off tools, each one takes the agent_name as input and provide the details of which agent to transfer to. We will use the pre-built `create_handoff_tool` for the lab. If you like to customize the swarm behaviour, you can customize the agent or create custom tool. Refer to this [link](https://github.com/langchain-ai/langgraph-swarm-py?tab=readme-ov-file#customizing-handoff-tools) for more details 

In [ ]:
transfer_to_existing_assistant = create_handoff_tool(
    agent_name="existing_mortgage_agent",
    description="Transfer user to the existing mortgage agent to provide information about current mortgage  details including outstanding principal, interest rates, maturity dates, payment schedules, and upcoming payment information. Use this when the user is asking about their existing mortgage account, payment details, or loan status."
)


transfer_to_application_assistant = create_handoff_tool(
    agent_name="mortgage_application_agent",
    description="Transfer user to the mortgage application agent to provide assistance with new mortgage applications, document status tracking, application details, and historical mortgage rate information. Use this when the user is asking about applying for a new mortgage, checking application status, or needs information about current mortgage rates."
)

For the multi agent architecture , we have seen different patterns already. Supervisor, 

In [ ]:
def load_system_prompt(filename):
    """Load system prompt from markdown file"""
    with open(f'config/{filename}', 'r') as f:
        return f.read()

In [ ]:
# Define the Agent
existing_mortgage_agent = create_react_agent(
    model=model,
    tools=[get_mortgage_details,transfer_to_application_assistant],
    prompt=dedent(load_system_prompt("existing_mortgage_system.md")),
    name="existing_mortgage_agent"
)


# Define the Agent
mortgage_application_agent = create_react_agent(
    model=model,
    tools=[get_application_details,get_mortgage_rate_history,get_mortgage_app_doc_status,transfer_to_existing_assistant],
    prompt=dedent("mortgage_application_system.md"),
    name="mortgage_application_agent")

In [ ]:
graph = create_swarm([existing_mortgage_agent,mortgage_application_agent],default_active_agent="existing_mortgage_agent")

app = graph.compile(checkpointer=memory)
config = {"configurable": {"thread_id": random.randint(1,10000)}}

display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
response = app.invoke(
    {"messages": "Hey, I am would like to open a new mortgage application, but before that I am interested to know current mortgage rates. Here is my id: 123456"},
    config
)

for message in response["messages"]:
    # prettify the output by adding message type and message
    message.pretty_print()
    # print(f"{message.type}: {message.content}")


In [ ]:
response = app.invoke(
    {"messages": "Hmm, probably what's 30 year fixed interest rate"},
    config
)

for message in response["messages"]:
    message.pretty_print()


Now that we created the agents with tool functions, lets add a RAG and add to the graph. We shall use Bedrock's Knowledge Base , the managed RAG offering. Replace the `knowledge_base_id` with the KB you created in the previous labs

For this scenario we will use  `AmazonKnowledgeBasesRetriever` class from  the langchain_aws module. 

<div class="alert alert-block alert-info">
Replace the KB id from the Bedrock modules
</div>

In [ ]:
from langchain_aws.retrievers import AmazonKnowledgeBasesRetriever

retriever = AmazonKnowledgeBasesRetriever(
                knowledge_base_id="EVDPJLTSOH",
                retrieval_config={
                    "vectorSearchConfiguration": {
                        "numberOfResults": 4
                    }
                
                },
            )



Now, we test to see if the KB is working as expected

In [ ]:
retriever.invoke("What is the benefit of refinancing, if any?")

Once we create the retriever, lets wrap it using the langchain `Tool`. This will help LangGraph to automatically present the KB as a tool

In [ ]:
from langchain.tools import Tool

retriever_tool = Tool(
    name="amazon_knowledge_base",
    description="Use this knowledge base to answer general questions about mortgages, like how to refinnance, or the difference between 15-year and 30-year mortgages.",
    func=lambda query: "\n\n".join([doc.page_content for doc in retriever.invoke(query)])
)




We add the 3rd handoff tool for the `general_mortgage_agent`

In [ ]:
transfer_to_existing_assistant = create_handoff_tool(
    agent_name="existing_mortgage_agent",
    description="Transfer user to the existing mortgage agent to provide information about current mortgage  details including outstanding principal, interest rates, maturity dates, payment schedules, and upcoming payment information. Use this when the user is asking about their existing mortgage account, payment details, or loan status. "
)


transfer_to_application_assistant = create_handoff_tool(
    agent_name="mortgage_application_agent",
    description="Transfer user to the mortgage application agent to provide assistance with new mortgage applications, document status tracking, application details, and historical mortgage rate information. Use this when the user is asking about applying for a new mortgage, checking application status, or needs information about current mortgage rates."
)

transfer_to_general_mortgage_agent = create_handoff_tool(
    agent_name="general_mortgage_agent",
    description="Transfer user to the general mortgage agent to use knowledge base to answer general questions about mortgages, like how to refinnance, or the difference between 15-year and 30-year mortgages."

)

Once the tools are created, we create `general_mortgage_agent` and re-create the existing agents with the `transfer_to_general_mortgage_agent` tool

In [ ]:
# Define the Agent
general_mortgage_agent = create_react_agent(
    model=model,
    tools=[retriever_tool,transfer_to_existing_assistant,transfer_to_application_assistant],
    prompt=dedent(load_system_prompt("general_assistant_system.md")),
    name="general_mortgage_agent")

# Define the Agent
existing_mortgage_agent = create_react_agent(
    model=model,
    tools=[get_mortgage_details,transfer_to_application_assistant, transfer_to_general_mortgage_agent],
    prompt=dedent(load_system_prompt("existing_mortgage_system.md")),
    name="existing_mortgage_agent"
)

# Define the Agent
mortgage_application_agent = create_react_agent(
    model=model,
    tools=[get_application_details,get_mortgage_rate_history,get_mortgage_app_doc_status,transfer_to_existing_assistant,transfer_to_general_mortgage_agent],
    prompt=dedent("mortgage_application_system.md"),
    name="mortgage_application_agent")


And we re-initiate the <b>swarm</b> multi agent setup

In [ ]:
graph = create_swarm([existing_mortgage_agent,mortgage_application_agent,general_mortgage_agent],default_active_agent="general_mortgage_agent")

app = graph.compile(checkpointer=memory)
config = {"configurable": {"thread_id": random.randint(1,10000)}}

display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:

response = app.invoke(
    {"messages": "I need a complete mortgage consultation. First, I want to check my existing mortgage details for customer ID xyz to see my current situation. Then I need to understand the market by showing me the 30-year fixed mortgage rate history for the past 30 days to compare with my current rate. Finally, I have a general question about mortgage refinancing - what are the key factors I should consider when deciding whether to refinance my mortgage, and what's the difference between a 15-year and 30-year mortgage in terms of total cost"},
    config
)

for message in response["messages"]:
    message.pretty_print()

Optionally, you could track the ai_messages and tool_messages separately

```
ai_messages = [msg for msg in response["messages"] if isinstance(msg, AIMessage)]
tool_messages = [msg for msg in response["messages"] if isinstance(msg, ToolMessage)]

for message in ai_messages:
    message.pretty_print()
```

## Summary

In this lab, we explored how to build a multi-agent system using LangGraph to create a comprehensive mortgage assistance solution. We built upon the single-agent concepts from the previous lab and implemented a specialized agent network with distinct roles and responsibilities.

   - Implemented three specialized agents with distinct roles and  established clear boundaries of responsibility between agents 
   - Connected agents to relevant tools based on their specialization
   - Created  handoff mechanism between agents
   - Demonstrated how specialized agents can provide more focused responses
   - Showed how complex customer inquiries can be routed to the most appropriate agent
   - Illustrated how multi-agent systems can handle a wider range of tasks than single agents

